In [1]:
#######################
# Project 3
# ESE 572
# Layer 2 Channel Coding
#######################

# Choose a probability of bit error to use when simulating the receiving end.

prob = 0.1
# prob = 0.01

In [2]:
#######################
# Step 1
#######################

# Function to convert string to binary using ASCII encoding
def string_to_binary(string):
    binary = ''
    binaryS = ''
    binaryC = ''
    counter = 0
    for char in string:
        ascii_code = ord(char) # Get ASCII code of character
        if counter == 24:
            binary += format(ascii_code, '08b')
            binaryS += format(ascii_code, '08b') + ' '
            binaryC += format(ascii_code, '08b') + '\n'
            counter = 0
        else:
            binary += format(ascii_code, '08b')
            binaryS += format(ascii_code, '08b') + ' '
            binaryC += format(ascii_code, '08b')
            counter += 1
    return binary, binaryS, binaryC

# Load the document
filename = 'input.txt'
text = ''
with open(filename,'r') as i:
    text += i.readline()
    
binary_data, binaryS_data, binaryC_data = string_to_binary(text)

# binary_data is single line
# binaryS_data contain spaces to delimitate each character
# binaryC_data is split as chunks of 200 bits


In [3]:
#######################
# Step 2
#######################

# Create CRC for g(D) = [D16 + D12 + D5 + 1]
crc_str = '10001000000100001' # divisor
int_crc_str = int(crc_str,2)
frames = []

binary_chunks = binaryC_data.split('\n')
for chunk in binary_chunks[:-1]:
    temp = chunk + "0000000000000000"
    temp = int(temp,2)
    crc = temp % int_crc_str
    crc = format(crc, '016b')
    if len(crc) > 16:
        crc = crc[-16:]
    frames.append(chunk + crc)

print(len(frames[0]))

216


In [4]:
#######################
# Step 3
#######################

import numpy as np

def encode_message(msg, matrix):
    code = np.mod(np.dot(msg, matrix), 2)
    return code

def decode_message(code, matrix):
    syndrome = np.mod(np.dot(code, matrix.T), 2)
    if np.count_nonzero(syndrome) == 0:
        # No errors detected
        return code[:4]
    else:
        # Single error detected, correct it
        error_pos = np.dot(syndrome, 2 ** np.arange(syndrome.size))
        corrected_code = code.copy()
        corrected_code[error_pos] = (corrected_code[error_pos] + 1) % 2
        return corrected_code[:4]

en = np.array([[1,1,1,1,1,1,1,1],
              [0,0,0,0,1,1,1,1],
              [0,0,1,1,0,0,1,1],
              [0,1,0,1,0,1,0,1]])

de = np.array([[0,1,0,1,0,1,0,1],
              [1,1,1,1,0,0,0,0],
              [1,1,0,0,1,1,0,0],
              [1,1,1,1,1,1,1,1]])

msg = np.array([1, 0, 1, 1])

code = encode_message(msg, en)
print("Encoded code:", code)
print()

received_code = np.array([0, 0, 0, 1, 1, 1, 1, 0])
decoded_msg = decode_message(received_code, de)
print("Decoded message:", decoded_msg)


def fec_string(input_string):
    output_string = ""
    for char in input_string:
        if char != '\n':
            output_string += char * 3
        else:
            output_string += char
    return output_string

binaryFEC = []
for frame in frames:
    binaryFEC.append(fec_string(frame))
    
print(len(binaryFEC[0]))

Encoded code: [1 0 0 1 1 0 0 1]

Decoded message: [0 0 1 1]
648


In [5]:
#######################
# Step 4
#######################

# flag = '01111110'
flag = '000111111111111111111000' # tripled
framed_frames = []
bitstuff=0
for binFEC in binaryFEC:
    i = 0
    
    while(i <= (len(binFEC) - 18)):
        if binFEC[i:i+17] == '11111111111111111':
            binFEC = binFEC[:i+16] + '0' + binFEC[i+16:] # bit stuff after any string of five 1s
            bitstuff+=1
        i += 3
    framed_frames.append(flag + binFEC)
    
framed_frames.append(flag) # after last FEC frame, insert another flag
print('bits stuffed: '+str(bitstuff))
print(framed_frames[0])

bits stuffed: 6
0001111111111111111110000001110000001110000001110001111110001111111110000000001110000000000000000000001111110000000001110000001111111110000000000000001111110001110001110000001111110000001111110000001110001111110000000000001110000000000000000001111111110000000000000001111111110000001110000001111110001111111111110001111110001111110001110001111110001110000001110001111110001111111110000001111110000001110001110001111110001111111110000001111111110001110000000000001110000000000000000001110001110000001111110001111111110001110000000000001110001111111110000000001110000000000000000001110000001111110000000001111110001111111111111111011111111111000000000111000111111111000111000


In [6]:
#######################
# Sequence to Transmit
#######################

sequence = ''
for fram in framed_frames:
    sequence += fram
    # print(len(fram))
    
print(len(sequence))

30270


In [7]:
#######################
# Step 5: Error
#######################

import random

lim = {0.1: 9,
       0.01: 99}

rxbits = ''
un = 0

for bit in sequence:
    rnum = random.randint(0,lim[prob])
    if rnum == 0: # error bit
        un += 1
        if bit == '1':
            bit = '0'
        else:
            bit = '1'
    rxbits += bit
print(un)

3024


In [8]:
#######################
# Step 5: Unstuffing
#######################

# stuff_seq = '000111110'

def unstuff(bits):
    i = 0
    while i < len(bits)-3:
        #print(bits[i:i+19])
        if bits[i:i+18] == '111111111111111101':
            print(bits[i:i+18])
            bits = bits[:i+16] + bits[i+17:] # remove bit stuffing
            print(bits[i:i+18])
            i += 19
        else:
            i += 3
    return bits

rxbits = unstuff(rxbits)
# rxbits, un = unstuff(sequence)

In [9]:
#######################
# Step 5: Fixing
#######################

def fix_errors(bits):
    i = 0
    un = 0
    while i < len(bits)-2:
        if bits[i] != bits[i+1]:
            un += 1
            # second bit wrong
            if bits[i] == bits[i+2]:
                bits[i+1] = bits[i]
            else:
                # first bit wrong
                bits[i] = bits[i+1]
        else:
            if bits[i+1] != bits[i+2]:
                # third bit wrong
                un += 1
                bits[i+2] = bits[i]
        i += 3
    return bits, un

cor_bits, un = fix_errors(list(rxbits))
print('BITS ERRORS : ' + str(un))
corrected_bits = ''

for b in cor_bits:
    corrected_bits += b
    
# print(corrected_bits)
# incorrect bit flips should be fixed

# finding flags to reseparate frames
i = 0
start = -1
end = -1
new_frames = []
flag_flag = 0

while (i < len(corrected_bits)):
    if corrected_bits[i:i+24] == flag:
        flag_flag = 1
        if start != -1:
            end = i - 1
            temp = corrected_bits[start:end]
            new_frames.append(temp)
            start = i
        else:
            temp = corrected_bits[:i]
            new_frames.append(temp)
            start = i
    if flag_flag:
        i += 24
        flag_flag = 0
    else:
        i += 3
new_frames.append(corrected_bits[start:])

print(new_frames[0])
# frames separated, unstuffed but need :: untripled, CRC checked, and then converted to text

BITS ERRORS : 4949



In [10]:
#######################
# Step 5: Un - triple
#######################

def anti_triplet(frame):
    compressed_frame = ''
    i = 0
    while i < len(frame):
        if i+2 < len(frame) and frame[i] == frame[i+1] == frame[i+2]:
            compressed_frame += frame[i]
        i += 3
    return compressed_frame

rxseqs = []
for nf in new_frames:
    rxseqs.append(anti_triplet(nf))

rxseqs = rxseqs[1:-1]
print(len(rxseqs[-1]))
print(rxseqs[2])
print(rxseqs[0])
# need :: CRC checked, and then converted to text

224
011111100110110011110001001110110010000001101100011001010111001001100011111010001110000101101111001101000001000001010111011010011111110101101101011000010100111100110000010000010111001001101011011100110010000000101101110001000
01111110010010010110111010100000001100010011110000110101001100110010110000101000011100001111001001101111011001010110100101101110011001010110111001110100001000000101001101110100001011100010000001001100011011


In [11]:
#######################
# Step 5: Remove flag
#######################
i = 0
for rxs in rxseqs:
    j=0
    rxs = rxs[8:]
    while j <len(rxs):
        if rxs[j:j+8]=='01111110':
            rxs=rxs[j+8:]
            print('here')
        j+=1
    rxseqs[i]=rxs
    i+=1
    
#print(rxseqs)

In [12]:
#######################
# Step 5: Check CRC
#######################

# Create CRC for g(D) = [D16 + D12 + D5 + 1]
crc_str = '10001000000100001' # divisor
int_crc_str = int(crc_str,2)
failed = 0
total = 0

for rxs in rxseqs:
    crc_rx = rxs[-16:]
    beginning = rxs[:-16] + "0000000000000000"
    beginning = int(beginning,2)
    crc = beginning % int_crc_str
    crc = format(crc,'016b')
    if len(crc) > 16:
        crc = crc[-16:]
    if crc != crc_rx:
        print("crc: "+str(crc)+" crc_rx: "+str(crc_rx))
        failed += 1
    total += 1

print('Number of Failed Frames with Error Probability ' + str(prob) + ': --- ' + str(failed) + ' --- \n')
print('Failed Frames: ' + str((failed / total) * 100) + '%' + ' of ' + str(total) + ' total frames recieved.')

crc: 1110010111101111 crc_rx: 0001001100011011
crc: 0000000000000000 crc_rx: 0010111000
crc: 1110100010100110 crc_rx: 0101101110001000
crc: 1110010011011010 crc_rx: 1100001100000101
crc: 0011001111101101 crc_rx: 1001001100101001
crc: 1100110000100111 crc_rx: 1111001010100011
crc: 1001011100100000 crc_rx: 0011111000111100
crc: 0110100111100110 crc_rx: 0111111100110101
crc: 0000011111111111 crc_rx: 0111010011001101
crc: 1000001010010011 crc_rx: 0110110101101110
crc: 1011101011101010 crc_rx: 0001101011100110
crc: 1100001001101011 crc_rx: 1110011000100111
crc: 1010110111010110 crc_rx: 1100110000010111
crc: 0010001000101010 crc_rx: 0010010000011111
crc: 0001101010000000 crc_rx: 1111101110010010
crc: 1111001101000010 crc_rx: 1110011110001100
crc: 1010001011100100 crc_rx: 0101011001000110
crc: 1000011011000000 crc_rx: 1000110001101011
crc: 1000000000101010 crc_rx: 1110011101110000
crc: 1010101101110001 crc_rx: 0010010000001111
crc: 1110000000100010 crc_rx: 0100111000100110
crc: 01010111000100

In [13]:
#######################
# Convert Back to Text
#######################

all_rx_bits = ''
char_arr = []
paragraph = ''
i = 0

for rxs in rxseqs:
    # print(rxs)
    if len(rxs[:-15]) < 200:
        continue
    else:
        all_rx_bits += rxs[:-15]
    
while i < len(all_rx_bits)-7:
    char_arr.append(all_rx_bits[i:i+8])
    i += 8

for cb in char_arr:
    character = chr(int(cb,2))
    if (character!='~'):
        paragraph += character

print(paragraph)

 [7ù½Éµýµ ¥5¥Í½ÔÉ¤ÊÈÒFÂØ@ÞØØÊÎÊKÒÜ@`p2r\


In [14]:
# In 1853, prominent St. Louis merchant Wayman Crow and his pastor, William Greenleaf Eliot Jr., concerned about 
# the lack of institutions of higher learning in the growing midwest, led the founding of Washington University  
# in St. Louis. During the 1840s and 50s, waves of immigrants flooded into St. Louis, boosting the population of 
# the young city. With these newcomers came a pressing need for education - both industrial training and basic 
# general courses - conducted outside of normal working hours. So the first educational step of the young 
# Washington University was to establish an evening program on October 22, 1854. Over the succeeding decades, 
# the continuing education program underwent many changes. The university flourished at its location in downtown 
# St. Louis for its first 50 years, growing from an evening program to an institution offering a full slate of 
# scientific, liberal arts and classical course offerings. In time, schools of law and fine arts were added. In 
# 1891, the school acquired the St. Louis Medical College to form a medical department, which merged with the 
# Missouri Medical College in 1899.